In [1]:
!pip install eng_to_ipa
import eng_to_ipa as ipa

def is_vow(character):
    '''
    Is the given (lowercase) character a vowel or not.
    '''
    #ipa_vowels = "iɪeɛæuʊoɔɑəʌ"
    ipa_vowels = "yøœɶɒɔoʊuʉiɪeɛæaɐɑʌɤɯɨɜ"
    return character in ipa_vowels

def is_space(character):
    '''
    Is the given character a space or newline (other space characters are
    cleaned in the preprocessing phase).
    '''
    return character==' ' or character=='\n'

def get_phonetic_transcription(lyrics):
    lines = lyrics.splitlines()
    phonetic_lines = [ipa.convert(line) for line in lines]
    phonetic = "\n".join(phonetic_lines)

    return phonetic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eng_to_ipa: filename=eng_to_ipa-0.0.2-py3-none-any.whl size=2822607 sha256=6ec9f628ff2a1ea95569cd112775a27aace77dd6790126d3c068e5e668efe58f
  Stored in directory: /root/.cache/pip/wheels/5b/ab/07/fe6722f710d8ef8bd0ccb4eb689ef96f5552f3fc0c80c1aa9c
Successfully built eng_to_ipa


In [24]:
import re
import numpy as np
import phonetics as ph
import syllables

def syllable_similarity(bar1, bar2):
    similarity = abs(syllables.estimate(bar1) - syllables.estimate(bar2))
    return similarity

def get_syllable_count_difference(lyrics):
    lines = lyrics.splitlines()
    i = 0
    similarity = 0
    while i < len(lines)-1:
        similarity += syllable_similarity(lines[i], lines[i+1])
        i += 2
    return similarity/len(lines)

def get_rhyme_density(lyrics, lookback=15):
    bars = Lyrics(lyrics, lookback)
    return bars.avg_rhyme_length

def get_longest_rhyme(lyrics, lookback=15):
    bars = Lyrics(lyrics, lookback)
    return bars.get_longest_rhyme()[0]

def get_unique_words(lyrics):
    words = lyrics.split()
    unique_words = set(words)
    return len(unique_words)/len(words)

def print_lyrics_stats(lyrics, lookback=15, artist=None, title=None):
    bars = Lyrics(lyrics, lookback, artist, title)
    print(bars.title)
    print('------------------------------------------')
    print("Average rhyme length: %.3f\n" % bars.avg_rhyme_length)
    bars.print_rhyme(bars.longest_rhyme)
    print("Average syllable count difference: %.3f\n" % get_syllable_count_difference(lyrics))
    print("Percentage unique words: %.3f\n" % get_unique_words(lyrics))




class Lyrics:
    '''
    This class is used to store and preprocess rap lyrics and calculate
    statistics like average rhyme length out of the lyrics.
    '''

    def __init__(self, text=None, lookback=15, artist=None, title=None):
        '''
        Lyrics can be read from the file (default) or passed directly
        to this constructor.
        '''
        self.text_raw = None
        # How many previous words are checked for a rhyme.
        self.lookback = lookback
        self.text_raw = text
        if artist == None or title == None:
            self.title = "Generated Song"
        else:
            self.title = title + " by " + artist

        if self.text_raw is not None:
            cleaning_ok = self.clean_text(self.text_raw)
            self.compute_vowel_representation()
            self.avg_rhyme_length, self.longest_rhyme = self.rhyme_stats()

    def clean_text(self, text):
        '''
        Preprocess text by removing unwanted characters and duplicate rows.
        '''

        self.text = text
        # If there are more than 2 consecutive newlines, remove some of them
        # (just to make the cleaned text look prettier)
        self.text = re.sub('\n\n+', '\n\n', self.text)
        # Remove duplicate rows
        self.lines = self.text.split('\n')

        uniq_lines = set()
        new_text = ''
        for l in self.lines:
            l = l.strip()
            if len(l) > 0 and l in uniq_lines:
                continue
            # Remove lines that are within brackets/parenthesis
            if len(l) >= 2 and ((l[0]=='[' and l[-1]==']') or (l[0]=='(' and l[-1]==')')):
                continue
            uniq_lines.add(l)
            new_text += l + '\n'

        self.text = new_text

    def compute_vowel_representation(self):
        '''
        Compute a representation of the lyrics where only vowels are preserved.
        '''
        self.vow = [] # Lyrics with all but vowels removed
        self.vow_idxs = [] # Indices of the vowels in self.text list
        self.word_ends = [] # Indices of the last characters of each word
        self.words = [] # List of words in the lyrics
        self.line_idxs = []

        self.text_orig = self.text
        self.text = get_phonetic_transcription(self.text)
        self.word_ends_orig = []
        self.words_orig = []

        prev_space_idx = -1 # Index of the previous space char
        line_idx = 0 # Line index of the current character
        # Go through the lyrics char by char
        for i in range(len(self.text)):
            self.line_idxs.append(line_idx)
            c = self.text[i]
            #c = ph.map_vow(c)
            if is_vow(c):
                # Ignore double vowels
                # (in English this applies probably only to 'aa' as in 'bath'
                # which rhymes with 'trap' that has only 'a')
                if i > 0 and self.text[i-1] == c:
                    # Index of a double vowel points to the latter occurrence
                    self.vow_idxs[-1] = i
                    continue
                # TODO Diftongs should not be split (i.e. "price" should
                # not rhyme with "trap kit"). This has been fixed in BattleBot
                self.vow.append(c)
                self.vow_idxs.append(i)
            elif is_space(c):
                if c in '\n':
                    line_idx += 1
                elif c in '.!?' and i < len(self.text)-1 and self.text[i+1] != '\n':
                    line_idx += 1
                # If previous char was not a space, we've encountered word end
                if len(self.vow) > 0 and not is_space(self.text[i-1]):
                    # Put together the new word. Potential consonants in the
                    # end are ignored
                    new_word = self.text[prev_space_idx+1:self.vow_idxs[-1]+1]
                    # Check that the new word contains at least one vowel
                    no_vowels = True
                    for c2 in new_word:
                        if is_vow(c2):
                            no_vowels = False
                            break
                    if no_vowels:
                        prev_space_idx = i
                        continue
                    self.word_ends.append(len(self.vow)-1)
                    self.words.append(new_word)
                prev_space_idx = i

        self.lines_orig = self.text_orig.split('\n')

    def rhyme_length(self, wpos2):
        '''
        Length of rhyme (in vowels). The latter part of the rhyme ends with
        word self.words[wpos2].

        Input:
            wpos2       Word index of the end of the rhyme.
        '''
        max_length = 0
        max_wpos1 = None
        wpos1 = max(0,wpos2-self.lookback)
        while wpos1 < wpos2:
            rl = self.rhyme_length_fixed(wpos1, wpos2)
            if rl > max_length:
                max_length = rl
                max_wpos1 = wpos1
            wpos1 += 1
        return max_length, max_wpos1

    def rhyme_length_fixed(self, wpos1, wpos2):
        '''
        Length of rhyme (in vowels). The first part of the rhyme ends with
        self.words[wpos1] and the latter part with word self.words[wpos2].

        Input:
            wpos1       Word index of the last word in the first part of the rhyme.
            wpos2       Word index of the end of the rhyme.
        '''
        if wpos1 < 0: # Don't wrap
            return 0
        elif self.words[wpos1] == self.words[wpos2]:
            return 0
        # Indices in the vowel list
        p1 = self.word_ends[wpos1]
        p2 = self.word_ends[wpos2]
        l = 0
        while self.vow[p1-l] == self.vow[p2-l]:
            # Make sure that exactly same words are not used
            if wpos1 > 0 and p1-l <= self.word_ends[wpos1-1] and wpos2 > 0 and p2-l <= self.word_ends[wpos2-1]:
                # Get the first and last character indices of the words surrounding the vowels at p1-l and p2-l
                prev_s1 = self.vow_idxs[p1-l]
                while prev_s1 > 0 and not is_space(self.text[prev_s1-1]):
                    prev_s1 -= 1
                prev_s2 = self.vow_idxs[p2-l]
                while prev_s2 > 0 and not is_space(self.text[prev_s2-1]):
                    prev_s2 -= 1
                next_s1 = self.vow_idxs[p1-l]
                while next_s1 < len(self.text)-1 and not is_space(self.text[next_s1+1]):
                    next_s1 += 1
                next_s2 = self.vow_idxs[p2-l]
                while next_s2 < len(self.text)-1 and not is_space(self.text[next_s2+1]):
                    next_s2 += 1
                if next_s1-prev_s1 == next_s2-prev_s2 and self.text[prev_s1:next_s1+1] ==  self.text[prev_s2:next_s2+1]:
                    break

            l += 1
            if p1-l < 0 or p2-l <= p1:
                break
        # Ignore rhymes with length 1
        if l == 1:
            l = 0
        return l

    def rhyme_stats(self):
        '''
        Compute the average rhyme length of the song and the longest rhyme.

        Output:
            Average rhyme length (float)
            Longest rhyme which is a 3-tuple with:
                (length, word index of the first part of the rhyme,
                         word index of the latter part of the rhyme)
        '''
        # Rhyme length of each word
        rls = []
        # Keep track of the longest rhyme
        max_rhyme = (0,None,None)
        for wpos2 in range(1,len(self.word_ends)):
            (rl, wpos1) = self.rhyme_length(wpos2)
            rls.append(rl)
            if rl > max_rhyme[0]:
                max_rhyme = (rl, wpos1, wpos2)
        rls = np.array(rls)
        # Average rhyme length of the song
        if len(rls) > 0:
            avg_rl = np.mean(rls)
        else:
            avg_rl = 0
        return avg_rl, max_rhyme

    def get_avg_rhyme_length(self):
        return self.avg_rhyme_length

    def print_song_stats(self):
        print('------------------------------------------')
        print("Avg rhyme length: %.3f\n" % self.avg_rhyme_length)

        self.print_rhyme(self.longest_rhyme)


    def print_rhyme(self, rhyme_tuple):
        print(self.get_rhyme_str(rhyme_tuple))

    def get_rhyme_str(self, rhyme_tuple):
        '''
        Construct a string of a given rhyme tuple.
        '''
        ret = ''
        rl, wpos1, wpos2 = rhyme_tuple
        if wpos1 is None or wpos2 is None:
            return ''
        p2 = self.vow_idxs[self.word_ends[wpos2]]
        p2_orig = p2
        # Find the ending of the last word
        while not is_space(self.text[p2]):
            p2 += 1
        p0 = self.vow_idxs[self.word_ends[wpos1]-rl]
        p0_orig = p0
        # Find the beginning of the line
        while self.text[p0] != '\n' and p0 > 0:
            p0 -= 1

        cap_line = ''
        rw1, rw2 = self.get_rhyming_vowels(rhyme_tuple)
        for i in range(p0,p2+1):
            if i == min(rw1) or i == min(rw2):
                cap_line += ' | ' + self.text[i]
            elif i == max(rw1) or i == max(rw2):
                cap_line += self.text[i] + '|'
            else:
                cap_line += self.text[i]
        ret += "Longest rhyme (l=%d): %s\n" % (rl, cap_line)
        # Get the corresponding lines from the original lyrics
        line_beg = self.line_idxs[p0]
        line_end = self.line_idxs[p2]
        for i in range(line_beg, line_end+1):
            if i < len(self.lines_orig):
                ret += self.lines_orig[i] + '\n'
        return ret

    def get_longest_rhyme(self):
        rhyme_str = self.get_rhyme_str(self.longest_rhyme)
        return self.longest_rhyme[0], rhyme_str

    def get_rhyming_vowels(self, rhyme_tuple):
        '''
        Return the indices of the rhyming vowels of the longest rhyme.

        Output:
            Tuple with the indices of the first part and the second part of
            the rhyme separately.
        '''
        rl, wpos1, wpos2 = rhyme_tuple
        if wpos1 is None or wpos2 is None:
            return ([-1],[-1])

        # The first part of the rhyme
        rhyme_idxs1 = [] # Indices of the rhyming vowels
        n_caps = 0
        p = self.vow_idxs[self.word_ends[wpos1]]
        while n_caps < rl:
            if is_vow(self.text[p]):
                rhyme_idxs1.append(p)
                # Increase the counter only if the vowel is not a double vowel
                if self.text[p] != self.text[p+1]:
                    n_caps += 1
            p -= 1

        # The second part of the rhyme
        rhyme_idxs2 = [] # Indices of the rhyming vowels
        n_caps = 0
        p = self.vow_idxs[self.word_ends[wpos2]]
        p_last = p
        while n_caps < rl:
            if is_vow(self.text[p]):
                rhyme_idxs2.append(p)
                # Increase the counter only if the vowel is not a double vowel.
                # The last vowel must be always counted.
                if p == p_last or self.text[p] != self.text[p+1]:
                    n_caps += 1
            p -= 1

        return (rhyme_idxs1, rhyme_idxs2)

In [4]:

import numpy as np
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import bigrams, ngrams, everygrams
from nltk.lm.preprocessing import padded_everygram_pipeline, pad_both_ends
from nltk.lm import MLE, KneserNeyInterpolated, Lidstone, Laplace, AbsoluteDiscountingInterpolated


In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df
train_df.rename({"Lyrics": "lyric", "Title": "song", "Artist": "artist"}, axis=1, inplace=True)
test_df.rename({"Lyrics": "lyric", "Title": "song", "Artist": "artist"}, axis=1, inplace=True)

## Tokenization

Default word tokenizer removes new line symbols.

In [4]:
# # Test
# lyric = [word_tokenize(line) + ['\n'] for line in train['lyric'][0].split('\n')]
# flat_list = list(itertools.chain(*lyric))[:-1]  # remove last new line symbol

In [8]:
def tokenize(dataset):
    return [
        nltk.flatten(([
            word_tokenize(line) + ['\n']
            for line in lyric.split('\n')
        ]))[:-1]
        for lyric in list(dataset['lyric'])
        if not isinstance(lyric, float)  # remove nan values
    ]

tokenized_train = tokenize(train_df)
tokenized_test = tokenize(test_df)

## Setup and stats

In [9]:
N = 3

In [10]:
def generate_ngrams(dataset, every=False):
    if every:
        return [list(everygrams(pad_both_ends(example, n=N), max_len=N)) for example in dataset]
    else:
        return [list(ngrams(pad_both_ends(example, n=N), n=N)) for example in dataset]

train_ngrams = generate_ngrams(tokenized_train, every=True)
test_ngrams = generate_ngrams(tokenized_test)

In [11]:
def generate_vocabulary(dataset):
    return list(nltk.lm.preprocessing.flatten(pad_both_ends(example, n=N) for example in dataset))

train_vocabulary = generate_vocabulary(tokenized_train)
test_vocabulary = generate_vocabulary(tokenized_test)

In [12]:
print("Training set contains", len(set(train_vocabulary)), "distinct tokens.")
print("Test set contains", len(set(test_vocabulary)), "distinct tokens.")
print("Test set has", len(set(test_vocabulary) - set(train_vocabulary)), "tokens not in train test.")

Training set contains 6757 distinct tokens.
Test set contains 3578 distinct tokens.
Test set has 1148 tokens not in train test.


In [13]:
from collections import Counter
Counter(train_vocabulary).most_common(10)

[('\n', 8628),
 (',', 5959),
 ('I', 3145),
 ('the', 2011),
 ("'", 1494),
 ('you', 1471),
 ('(', 1468),
 (')', 1468),
 ('a', 1204),
 ('it', 1164)]

## Training

In [14]:
lm = Lidstone(1e-4, N)
lm.fit(train_ngrams, train_vocabulary)

In [15]:
len(lm.vocab)

6758

## Quantitative evaluation

### Perplexity (PPL)

In [16]:
lm.perplexity(nltk.lm.preprocessing.flatten(train_ngrams))

34.729352351396486

In [17]:
lm.perplexity(nltk.lm.preprocessing.flatten(test_ngrams))

743.7346299905059

### Generate songs after first line

In [18]:
re.sub(r'[ ]+\n[ ]+', r'\n', ' '.join(lm.generate(100, text_seed=['I', 'found'])))

"\n And I 'll take , I 'm about\nHmm-hmm-hmm , hmm-hmm\nPalm to palm\nWe keep it like that , and like that , say my name , I ai n't your cup 's full of trap niggas ( Baow )\nCan I show you how I feel bad for you , yeah , yeah\nYou ai n't strong enough to take the lead\nI 'm in that sunshine state with a Speed Stick\n... </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>"

In [19]:
first_lines = [song[:song.index('\n')+1] for song in tokenized_test if '\n' in song]
len(first_lines)

110

In [20]:
first_lines[0]

['(', 'Ugh', ',', 'you', "'re", 'a', 'monster', ')', '\n']

Generate all continuations of rap songs

In [21]:
results = []
for first_line in first_lines:
    result = ' '.join(first_line)[:-2] + '\n'
    result += re.sub(r'[ ]+\n[ ]+', r'\n', ' '.join(lm.generate(100, text_seed=first_line, random_seed=0)))
    result = re.sub(' </s>.*', '', result)
    results.append(result)

In [65]:
print(results[8])

Traphouse Mob
Yoz , what , what I pay for hoes ( yeah )
Shawty go jogging every morning )
Bum bum be-dum bum ( I ca n't stress about no bitch 'cause I know
Ooh , that I 'd like to fuck I look around , niggas spinnin '

 Guess we all Pisces
Your last one standin ' in my hallways , I get down
All the ladies , they love smoking and love drinking , some love deep inside of the storm when it feels like it when you turn the music up


### Rhyme Density (RD)

In [25]:

def get_rhyme_density(lyrics, lookback=15):
    bars = Lyrics(lyrics, lookback)
    return bars.avg_rhyme_length
results_rd = np.array([get_rhyme_density(song) for song in results])

In [26]:
round(np.mean(results_rd),2)

0.66

In [27]:
round(np.std(results_rd),2)

0.21

### Syllable Count Difference (SCD)

In [29]:
results_scd = np.array([get_syllable_count_difference(song) for song in results])

In [30]:
round(np.mean(results_scd),2)

5.16

In [31]:
round(np.std(results_scd),2)

2.62

### Longest Rhyme (LR)

In [33]:
results_lr = np.array([get_longest_rhyme(song) for song in results])

In [60]:
print(round(np.mean(results_lr),2), round(np.std(results_lr),2))

3.61 0.84


### Unique Words (UW)

In [61]:
results_uw = np.array([get_unique_words(song) for song in results])

In [62]:
print(round(np.mean(results_uw),2), round(np.std(results_uw),2))

0.74 0.07


### Get those statistics for training and test sets

#### Train

In [35]:
train_rd = np.array([
    get_rhyme_density(song)
    for song in train_df['lyric']
    if isinstance(song, str) and '\n' in song])

In [36]:
round(np.mean(train_rd),2)

0.83

In [37]:
round(np.std(train_rd),2)

0.26

In [38]:
train_scd = np.array([
    get_syllable_count_difference(song)
    for song in train_df['lyric']
    if isinstance(song, str) and '\n' in song])

In [39]:
round(np.mean(train_scd),2)

2.28

In [40]:
round(np.std(train_scd),2)

0.66

In [41]:
train_lr = np.array([
    get_longest_rhyme(song)
    for song in train_df['lyric']
    if isinstance(song, str) and '\n' in song])

In [42]:
print(round(np.mean(train_lr),2), round(np.std(train_lr),2))

4.62 1.23


In [43]:
train_uw = np.array([
    get_unique_words(song)
    for song in train_df['lyric']
    if isinstance(song, str) and '\n' in song])

In [44]:
print(round(np.mean(train_uw),2), round(np.std(train_uw),2))

0.61 0.1


#### Test

In [45]:
test_rd = np.array([
    get_rhyme_density(song)
    for song in test_df['lyric']
    if isinstance(song, str) and '\n' in song])

In [46]:
round(np.mean(test_rd),2)

0.8

In [47]:
round(np.std(test_rd),2)

0.23

In [48]:
test_scd = np.array([
    get_syllable_count_difference(song)
    for song in test_df['lyric']
    if isinstance(song, str) and '\n' in song])

In [49]:
round(np.mean(test_scd),2)

2.21

In [50]:
round(np.std(test_scd),2)

0.7

In [51]:
test_lr = np.array([
    get_longest_rhyme(song)
    for song in test_df['lyric']
    if isinstance(song, str) and '\n' in song])

In [52]:
print(round(np.mean(test_lr),2), round(np.std(test_lr),2))

4.55 0.98


In [53]:
test_uw = np.array([
    get_unique_words(song)
    for song in test_df['lyric']
    if isinstance(song, str) and '\n' in song])

In [54]:
print(round(np.mean(test_uw),2), round(np.std(test_uw),2))

0.63 0.09


In [55]:
test_df

,Unnamed: 0,song,artist,lyric,split
0,1,Godzilla (feat. Juice WRLD),Eminem feat. Juice WRLD,"(Ugh, you're a monster)\n\nI can swallow a bot...",-0.945349
1,5,Rap God,Eminem,"Look, I was gonna go easy on you not to hurt y...",-1.382372
2,7,Space Bound,Eminem,"We touch, I feel a rush, we clutch, it isn't m...",-2.836138
3,8,Love The Way You Lie (Part II) - Pt. 2,Rihanna feat. Eminem,On the first page of our story\nThe future see...,0.259160
4,9,Love the Way You Lie (feat. Rihanna),Eminem feat. Rihanna,Just gonna stand there and watch me burn?\nWel...,0.758809
...,...,...,...,...,...
105,384,ASTROTHUNDER,Travis Scott,Yeah\n\nSeem like the life I fiend\nSeem like ...,0.056384
106,393,Pussy & Millions (feat. Travis Scott),Drake feat. 21 Savage & Travis Scott,"I been out, late night creepin', should I slid...",0.559218
107,395,CAN'T SAY,Travis Scott,No you can't say if I'm mad or not\nSmokin' he...,-0.919669
108,397,HYAENA,Travis Scott,The situation we are in at this time\nNeither ...,-0.644134


## Qualitative evaluation

In [56]:
print(re.sub(r'[ ]+\n[ ]+', r'\n', ' '.join(lm.generate(100, text_seed=['cut', 'the', 'music', 'up', '\n']))))

Thug niggas , a whole disc and
Nigga , my nigga said this
Now , girl , hey Nicki , hey ( yeah , yeah , yeah , yeah , yeah , yeah

 Y , baby

 She no Juliet for no good reason
Cause I know I 'm the man , looking for a nigga down with the movement , when they bang this in the coat
Anything to have a dog , call me a paper towel , gim me head
I 'm up in a drop-top , she know


In [57]:
print(re.sub(r'[ ]+\n[ ]+', r'\n', ' '.join(lm.generate(100, text_seed=['godfather', 'in', 'the', 'house', '\n']))))

... </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>


In [58]:
first_lines[0]

['(', 'Ugh', ',', 'you', "'re", 'a', 'monster', ')', '\n']

In [63]:
for i in range(10):
    print(f'### Sample number {seed+1} ###')
    print(' '.join(first_lines[i])[:-2])
    print(re.sub(r'[ ]+\n[ ]+', r'\n', ' '.join(lm.generate(100, text_seed=first_lines[i]))))
    print()

NameError: name 'seed' is not defined